***GENERATED CODE FOR Hospitalstay PIPELINE***

**CONNECTOR FUNCTIONS TO READ DATA FROM DATABRICKS FILESYSTEM**

In [ ]:
import datetime
import logging
import warnings
warnings.filterwarnings('ignore')
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.INFO)


class DBFSConnector:

    def fetch(inStages, inStagesData, stageId, spark, config):
        df = spark.read.\
            options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                    inferschema='true',
                    delimiter=eval(config)["delimiter"])\
            .csv(eval(config)['url'])
        display(df.limit(2).toPandas())
        return df

    def put(inStages, inStagesData, stageId, spark, config):
        return inStagesData.write.format('csv').options(header='true' if eval(config)["is_header"] == "Use Header Line" else 'false',
                                                        delimiter=eval(config)["delimiter"]).save(("%s %s") % (datetime.datetime.now().strftime("%Y-%m-%d %H.%M.%S")+"_", eval(config)['url']))


**TRANSFORMATIONS FUNCTIONS THAT WILL BE APPLIED ON DATA**

In [ ]:
import json
from pyspark.sql.types import IntegerType
from pyspark.ml.feature import StringIndexer
from pyspark.sql.functions import col, when
from pyspark.sql.types import IntegerType
from pyspark.sql.functions import mean, stddev, min, max, col


class CleanseData:
    # def __init__(self,df):
    #     #print()

    def replaceByMean(self, feature, df, mean_=-1):

        meanValue = df.select(mean(col(feature.name)).alias(
            'mean')).collect()[0]["mean"]
        df.fillna(meanValue, subset=[feature.name])
        df.withColumn(feature.name, when(col(feature.name) == " ",
                                         meanValue).otherwise(col(feature.name).cast("Integer")))
        return df

    def replaceByMax(self, feature, df, max_=-1):
        maxValue = df.select(max(col(feature.name)).alias('max')).collect()[
            0]["max"]
        df.fillna(maxValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", maxValue).otherwise(col(feature.name)))
        return df

    def replaceByMin(self, feature, df, min_=-1):
        minValue = df.select(min(col(feature.name)).alias('min')).collect()[
            0]["min"]
        df.fillna(minValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", minValue).otherwise(col(feature.name)))
        return df

    def replaceByStandardDeviation(self, feature, df, stddev_=-1):
        stddevValue = df.select(stddev(col(feature.name)).alias(
            'stddev')).collect()[0]["stddev"]
        df.fillna(stddevValue, subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", stddevValue).otherwise(col(feature.name)))
        return df

    def replaceDateRandomly(self, feature, df):
        fillValue = df.where(col(feature.name).isNotNull()
                             ).head(1)[0][feature.name]
        df.fillna(str(fillValue), subset=[feature.name])
        df = df.withColumn(feature.name,
                           when(col(feature.name) == " ", fillValue).otherwise(col(feature.name)))
        # print("CleanseData:replaceDateRandomly Schema : ", df.#printSchema())
        return df

    def replaceNullValues(self, fList, df):
        featuresList = df.schema.fields
        for featureObj in fList:
            for feat in featuresList:
                if featureObj["feature"] in feat.name:
                    featureName = feat
                    if "mean" in featureObj["replaceby"]:
                        df = self.replaceByMean(featureName, df)
                    elif "max" in featureObj["replaceby"]:
                        df = self.replaceByMax(featureName, df)
                    elif "min" in featureObj["replaceby"]:
                        df = self.replaceByMin(featureName, df)
                    elif "stddev" in featureObj["replaceby"]:
                        df = self.replaceByStandardDeviation(featureName, df)
                    elif "random" in featureObj["replaceby"]:
                        df = self.replaceDateRandomly(featureName, df)
        return df


def StringIndexerTransform(df, params):
    dfReturn = df
    feature = params["feature"]

    dfReturn = dfReturn.fillna({feature: ''})
    outcol = feature + "_transform"
    indexer = StringIndexer(
        inputCol=feature, outputCol=outcol, handleInvalid="skip")
    indexed = indexer.fit(dfReturn).transform(dfReturn)
    indexed = indexed.drop(feature).withColumnRenamed(outcol, feature)
    dfReturn = indexed
    distinct_values_list = dfReturn.select(
        feature).distinct().rdd.map(lambda r: r[0]).collect()
    len_distinct_values_list = len(distinct_values_list)
    if len_distinct_values_list <= 4:
        changed_type_df = dfReturn.withColumn(
            feature, dfReturn[feature].cast(IntegerType()))
        return changed_type_df
    # changed_type_df.show(3)
    return dfReturn


Feature_Transformations_Methods = {
    "String Indexer": StringIndexerTransform,

}


class TransformationMain:
    # TODO: change df argument in run with following
    def run(inStages, inStagesData, stageId, spark, config):
        configObj = json.loads(config)
        featureData = configObj["FE"]
        transformationDF = inStagesData[inStages[0]]
        transformationDF = CleanseData().replaceNullValues(featureData, transformationDF)
        for transformation in featureData:
            feature = transformation["feature"]
            if transformation["transformation"] != '' and transformation["selected"].lower() == "true" and not (feature.__contains__("_transform")):
                transformationDF = Feature_Transformations_Methods["%s" % transformation["transformation"]](
                    transformationDF, transformation)

        display(transformationDF.limit(2).toPandas())
        return transformationDF


**AUTOML FUNCTIONS**

In [ ]:
from tpot import TPOTClassifier
from sklearn.model_selection import train_test_split
import pyspark


def functionClassification(sparkDF, listOfFeatures, label):
    sparkDF.persist(pyspark.StorageLevel.MEMORY_AND_DISK)
    df = sparkDF.toPandas()
    df.columns.intersection(listOfFeatures)
    X = df.drop(label, axis=1).values
    y = df[label].values
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, random_state=1, test_size=0.1)
    tpotModel = TPOTClassifier(verbosity=3, n_jobs=-1, generations=10, max_time_mins=15,
                               population_size=15)
    tpotModel.fit(X_train, y_train)
    display(" Accuracy of Model : %s" % tpotModel.score(X_test, y_test))


**READING DATAFRAME**

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

try: 
	Hospitalstay_DBFS = DBFSConnector.fetch([], {}, "5ea717085290fb74ae38fd3a", spark, "{'url': '/Demo/PredictHospitalLengthOfStayTrain.csv', 'file_type': 'Delimeted', 'delimiter': ',', 'is_header': 'Use Header Line'}")

except Exception as ex: 
	logging.error(ex)


**TRANSFORMING DATAFRAME**

In [ ]:
try: 
	Hospitalstay_AutoFE = TransformationMain.run(["5ea717085290fb74ae38fd3a"],{"5ea717085290fb74ae38fd3a": Hospitalstay_DBFS}, "5ea717085290fb74ae38fd3b", spark,json.dumps( {"FE": [{"transformationsData": {}, "feature": "eid", "transformation": "", "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "20043", "mean": "50110.5", "stddev": "28847.88", "min": "2", "max": "100000", "missing": "0"}}, {"transformationsData": {"feature_label": "vdate"}, "feature": "vdate", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "20043", "mean": "", "stddev": "", "min": "01/01/2012", "max": "31/12/2012", "missing": "0"}, "transformation": "String Indexer"}, {"transformationsData": {"feature_label": "rcount"}, "feature": "rcount", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "20043", "mean": "0.91", "stddev": "1.29", "min": "0", "max": "5+", "missing": "0"}, "transformation": "String Indexer"}, {"transformationsData": {"feature_label": "gender"}, "feature": "gender", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "20043", "mean": "", "stddev": "", "min": "F", "max": "M", "missing": "0"}, "transformation": "String Indexer"}, {"transformationsData": {}, "feature": "dialysisrenalendstage", "transformation": "", "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "20043", "mean": "0.04", "stddev": "0.19", "min": "0", "max": "1", "missing": "0"}}, {"transformationsData": {}, "feature": "asthma", "transformation": "", "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "20043", "mean": "0.03", "stddev": "0.18", "min": "0", "max": "1", "missing": "0"}}, {"transformationsData": {}, "feature": "irondef", "transformation": "", "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "20043", "mean": "0.09", "stddev": "0.29", "min": "0", "max": "1", "missing": "0"}}, {"transformationsData": {}, "feature": "pneum", "transformation": "", "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "20043", "mean": "0.04", "stddev": "0.19", "min": "0", "max": "1", "missing": "0"}}, {"transformationsData": {}, "feature": "substancedependence", "transformation": "", "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "20043", "mean": "0.06", "stddev": "0.24", "min": "0", "max": "1", "missing": "0"}}, {"transformationsData": {}, "feature": "psychologicaldisordermajor", "transformation": "", "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "20043", "mean": "0.24", "stddev": "0.43", "min": "0", "max": "1", "missing": "0"}}, {"transformationsData": {}, "feature": "depress", "transformation": "", "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "20043", "mean": "0.05", "stddev": "0.22", "min": "0", "max": "1", "missing": "0"}}, {"transformationsData": {}, "feature": "psychother", "transformation": "", "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "20043", "mean": "0.05", "stddev": "0.22", "min": "0", "max": "1", "missing": "0"}}, {"transformationsData": {}, "feature": "fibrosisandother", "transformation": "", "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "20043", "mean": "0.01", "stddev": "0.07", "min": "0", "max": "1", "missing": "0"}}, {"transformationsData": {}, "feature": "malnutrition", "transformation": "", "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "20043", "mean": "0.05", "stddev": "0.22", "min": "0", "max": "1", "missing": "0"}}, {"transformationsData": {}, "feature": "hemo", "transformation": "", "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "20043", "mean": "0.08", "stddev": "0.27", "min": "0", "max": "1", "missing": "0"}}, {"transformationsData": {}, "feature": "hematocrit", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "20043", "mean": "11.99", "stddev": "2.04", "min": "4.4", "max": "23.9", "missing": "0"}, "transformation": ""}, {"transformationsData": {}, "feature": "neutrophils", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "20043", "mean": "10.22", "stddev": "5.43", "min": "0.1", "max": "245.9", "missing": "0"}, "transformation": ""}, {"transformationsData": {}, "feature": "sodium", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "20043", "mean": "137.9", "stddev": "3.0", "min": "125.2598136", "max": "151.3872826", "missing": "0"}, "transformation": ""}, {"transformationsData": {}, "feature": "glucose", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "20043", "mean": "141.9", "stddev": "30.05", "min": "-1.005926859", "max": "254.738735", "missing": "0"}, "transformation": ""}, {"transformationsData": {}, "feature": "bloodureanitro", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "20043", "mean": "14.2", "stddev": "13.48", "min": "1.0", "max": "682.5", "missing": "0"}, "transformation": ""}, {"transformationsData": {}, "feature": "creatinine", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "20043", "mean": "1.1", "stddev": "0.2", "min": "0.341221347", "max": "1.878477253", "missing": "0"}, "transformation": ""}, {"transformationsData": {}, "feature": "bmi", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "20043", "mean": "29.8", "stddev": "2.01", "min": "22.3206319", "max": "37.86556724", "missing": "0"}, "transformation": ""}, {"transformationsData": {}, "feature": "pulse", "transformation": "", "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "20043", "mean": "73.32", "stddev": "11.58", "min": "27", "max": "117", "missing": "0"}}, {"transformationsData": {}, "feature": "respiration", "type": "real", "selected": "True", "replaceby": "mean", "stats": {"count": "20043", "mean": "6.49", "stddev": "0.57", "min": "0.2", "max": "10.0", "missing": "0"}, "transformation": ""}, {"transformationsData": {}, "feature": "secondarydiagnosisnonicd9", "transformation": "", "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "20043", "mean": "2.1", "stddev": "2.01", "min": "0", "max": "10", "missing": "0"}}, {"transformationsData": {"feature_label": "discharged"}, "feature": "discharged", "type": "string", "selected": "True", "replaceby": "max", "stats": {"count": "20043", "mean": "", "stddev": "", "min": "01/01/2013", "max": "31/12/2012", "missing": "0"}, "transformation": "String Indexer"}, {"transformationsData": {}, "feature": "facilityid", "transformation": "", "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "20043", "mean": "2.75", "stddev": "1.65", "min": "1", "max": "5", "missing": "0"}}, {"transformationsData": {}, "feature": "number_of_issues", "transformation": "", "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "20043", "mean": "0.73", "stddev": "1.09", "min": "0", "max": "8", "missing": "0"}}, {"transformationsData": {}, "feature": "lengthofstay", "transformation": "", "type": "numeric", "replaceby": "mean", "selected": "True", "stats": {"count": "20043", "mean": "3.99", "stddev": "2.37", "min": "1", "max": "16", "missing": "0"}}, {"feature": "vdate_transform", "transformation": "", "transformationsData": {}, "type": "real", "selected": "True", "stats": {"count": "20043", "mean": "167.68", "stddev": "105.17", "min": "0.0", "max": "366.0", "missing": "0"}}, {"feature": "rcount_transform", "transformation": "", "transformationsData": {}, "type": "real", "selected": "True", "stats": {"count": "20043", "mean": "1.12", "stddev": "1.54", "min": "0.0", "max": "5.0", "missing": "0"}}, {"feature": "gender_transform", "transformation": "", "transformationsData": {}, "type": "numeric", "selected": "True", "stats": {"count": "20043", "mean": "0.42", "stddev": "0.49", "min": "0", "max": "1", "missing": "0"}}, {"feature": "discharged_transform", "transformation": "", "transformationsData": {}, "type": "real", "selected": "True", "stats": {"count": "20043", "mean": "168.72", "stddev": "105.43", "min": "0.0", "max": "374.0", "missing": "0"}}]}))

except Exception as ex: 
	logging.error(ex)


**TRAIN MODEL**

In [ ]:
try: 
	functionClassification(Hospitalstay_AutoFE, ["eid", "vdate", "rcount", "gender", "dialysisrenalendstage", "asthma", "irondef", "pneum", "substancedependence", "psychologicaldisordermajor", "depress", "psychother", "fibrosisandother", "malnutrition", "hemo", "hematocrit", "neutrophils", "sodium", "glucose", "bloodureanitro", "creatinine", "bmi", "pulse", "respiration", "secondarydiagnosisnonicd9", "discharged", "facilityid", "number_of_issues"], "lengthofstay")

except Exception as ex: 
	logging.error(ex)
